In [1]:
import pandas as pd
import requests, bs4,re
import numpy as np
from datetime import timedelta, date
import datetime
pd.set_option('float_format', '{:f}'.format)
pd.set_option('display.max_columns', None)
pd.options.display.max_colwidth = 100

In [2]:
transfers_det= pd.read_pickle("C:/Users/YAVUZ/Transfer Fee Project/Transfer Details/Transfers_Hst_Detail2.pkl")
transfers_det.reset_index(drop=True, inplace=True)

In [3]:
def team_played_scraper(start,end):
    df2=pd.DataFrame()
    for j in range(start,end):
        print(j)
        url='https://www.transfermarkt.com'+ transfers_det['Transfer_Link'][j]
        headers = {"User-Agent":"Mozilla/5.0"}
        response = requests.get(url, headers=headers)
        soup = bs4.BeautifulSoup(response.text, 'html.parser')
        table= soup.find_all('table')[0] # Transfer Details Table
        new_table = pd.DataFrame(columns=range(0,len(table.find_all('td'))), index =range(0,len(table.find_all('tr')))) 
        row_marker=0
        for row in table.find_all('tr'):
            column_marker = 0
            columns = row.find_all('td')
            for column in columns:
                if len(column.select('img'))>0:
                    new_table.iat[row_marker,column_marker] = column.select('img')[0].get('alt')
                else:
                    new_table.iat[row_marker,column_marker] = column.get_text()
                column_marker += 1
            row_marker+=1
            
        new_table=new_table.dropna(axis=1, how='all')
        new_table=new_table[new_table.loc[:,1].notnull()].loc[:,[1,6]]
        new_table=new_table.rename(columns={1:'Start',6:'Team'})
        new_table=new_table[new_table['Team'].notnull()]
        new_table.reset_index(drop=True, inplace=True)
        new_table['Start']=[ datetime.datetime.strptime(date, '%b %d, %Y') for date in new_table['Start']]
        new_table['End'] = new_table['Start'].shift(1)       
        new_table['Transfer_Link']= transfers_det['Transfer_Link'][j]
        new_table['Transfer_id'] = transfers_det['Transfer_id'][j]
        new_table['id'] = transfers_det['id'][j]
        new_table['Transfer_Date'] = transfers_det['Transfer_Date'][j]
        new_table=new_table[['Team','Start', 'End','Transfer_Link', 'Transfer_id', 'id', 'Transfer_Date']]
        df2=df2.append(new_table,ignore_index=True,sort=False)
    return df2

In [4]:
Team_played=team_played_scraper(0,500)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [5]:
Team_played.to_pickle("C:/Users/YAVUZ/Transfer Fee Project/Transfer Details/Team_played0_500.pkl")